In [ ]:
!pip install -q /kaggle/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install -q simpletransformers==0.51.0 --no-index --find-links=file:///kaggle/input/simpletransformers/simpletransformers-0.51.0

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv', 
                    usecols=['excerpt', 'target'])
train.columns = ['text', 'labels']
train_df, valid_df = train_test_split(train, test_size=0.01, random_state=42)

In [ ]:
model_args = ClassificationArgs()
model_args.max_seq_length = 300
model_args.num_train_epochs = 5
model_args.regression = True
model_args.no_save = True
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model = ClassificationModel(
    "roberta",
    "../input/robertalarge",
    num_labels=1,
    args=model_args
)

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(valid_df)
print(result)

In [ ]:
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv', 
                   usecols=['id', 'excerpt'])
test.columns = ['id', 'text']

predictions, _ = model.predict(test['text'].values)
test['target'] = predictions

test[['id', 'target']].to_csv('submission.csv', index=False)